In [1]:
#!apt-get install -y xvfb python-opengl > /dev/null 2>&1
#!pip install gym pyvirtualdisplay > /dev/null 2>&1
#!apt-get install x11-utils
!pip install keras-rl
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.13.1 

     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=15a7bf1c08c1d20ea19a4d341395a8e4e10b7a2d05bc6bbfb4eb92ca61ca83f5
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 4.9MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 36.7MB/s 
     |████████████████████████████████| 368kB 49.9MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
 

In [1]:
from __future__ import division
import gym
import atari_py
from PIL import Image
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
#print(atari_py.list_games())

print(keras.__version__)
print(tf.__version__)

Using TensorFlow backend.


2.2.4
1.13.1


In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('float32')/255 

    #def process_reward(self, reward):
     #   return np.clip(reward, -1., 1.)

In [3]:
#game = 'Freeway-v0' #3
#game = 'BeamRider-v0' #9
#game = 'Centipede-v0' #18
#game = 'DemonAttack-v0' #6
game = 'AirRaid-v0' #6

env = gym.make(game, full_action_space=True)
np.random.seed(123)
env.seed(123)
#env.reset()
nb_actions = env.action_space.n
#print(nb_actions)

In [4]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')
    
model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(nb_actions, activation = 'linear'))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dens

In [5]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=100000)

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn.compile(Adam(lr=.00025), metrics=['mae'])


In [24]:
dqn.load_weights('DQN_'+game+'_weights_0')
dqn.fit(env, nb_steps=200000, visualize=False, verbose=1)
dqn.save_weights('DQN_'+game+'_weights',overwrite=True)

Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.8125
14 episodes - episode_reward: 553.571 [150.000, 1325.000] - ale.lives: 0.999

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.8600
14 episodes - episode_reward: 605.357 [175.000, 1475.000] - ale.lives: 0.999

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.9325
20 episodes - episode_reward: 483.750 [225.000, 825.000] - ale.lives: 0.998

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 0.9900
24 episodes - episode_reward: 414.583 [150.000, 950.000] - ale.lives: 0.998

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 26s 3ms/step - reward: 0.8950
26 episodes - episode_reward: 344.231 [75.000, 1300.000] - ale.lives: 0.997

Interval 6 (50000 steps per

In [6]:
dqn.load_weights('DQN_'+game+'_weights')
history = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 2625.000, steps: 2401
Episode 2: reward: 5475.000, steps: 4418
Episode 3: reward: 4700.000, steps: 4096
Episode 4: reward: 2150.000, steps: 2512
Episode 5: reward: 2225.000, steps: 2544
Episode 6: reward: 3375.000, steps: 2970
Episode 7: reward: 2300.000, steps: 2709
Episode 8: reward: 3300.000, steps: 2990
Episode 9: reward: 2125.000, steps: 2864
Episode 10: reward: 2350.000, steps: 3006


In [7]:
env.close()

In [8]:
dqn.load_weights('DQN_'+game+'_weights')
history = dqn.test(env, nb_episodes=100, visualize=False)

from statistics import mean

print('Average Reward : {}'.format(mean(history.history['episode_reward'])))

Testing for 100 episodes ...
Episode 1: reward: 3575.000, steps: 2941
Episode 2: reward: 2150.000, steps: 2452
Episode 3: reward: 1775.000, steps: 2325
Episode 4: reward: 3525.000, steps: 3770
Episode 5: reward: 4175.000, steps: 3905
Episode 6: reward: 2850.000, steps: 2752
Episode 7: reward: 2500.000, steps: 2353
Episode 8: reward: 1750.000, steps: 2451
Episode 9: reward: 4875.000, steps: 4261
Episode 10: reward: 2525.000, steps: 2867
Episode 11: reward: 3000.000, steps: 2707
Episode 12: reward: 4575.000, steps: 3609
Episode 13: reward: 3775.000, steps: 3347
Episode 14: reward: 5500.000, steps: 4483
Episode 15: reward: 1650.000, steps: 2334
Episode 16: reward: 3525.000, steps: 3160
Episode 17: reward: 1150.000, steps: 1830
Episode 18: reward: 2500.000, steps: 2415
Episode 19: reward: 2775.000, steps: 3329
Episode 20: reward: 1375.000, steps: 2483
Episode 21: reward: 2525.000, steps: 2482
Episode 22: reward: 2975.000, steps: 2917
Episode 23: reward: 4325.000, steps: 3434
Episode 24: re